In [1]:
import pandas as pd
import numpy as np

In [2]:
beers_rb = pd.read_csv("dataset/RateBeer/beers.csv")
beers_ba = pd.read_csv("dataset/BeerAdvocate/beers.csv")
beers_matched = pd.read_csv("dataset/MatchedData/beers.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_6840\3015418705.py:3: DtypeWarning: Columns (0,1,2,3,4,5,8,10,11,12,13,15,16,17,18,19,20,23,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  beers_matched = pd.read_csv("dataset/MatchedData/beers.csv")


In [3]:
users_rb = pd.read_csv("dataset/RateBeer/users.csv")
users_ba = pd.read_csv("dataset/BeerAdvocate/users.csv")
users_matched = pd.read_csv("dataset/MatchedData/users.csv")

In [4]:
ratings_rb = pd.read_csv("dataset/RateBeer/ratings.csv")
ratings_ba = pd.read_csv("dataset/BeerAdvocate/ratings.csv")
ratings_matched = pd.read_csv("dataset/MatchedData/ratings.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_6840\3451628320.py:3: DtypeWarning: Columns (0,1,2,3,5,7,8,9,10,11,13,17,18,19,20,22,24,25,26,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_matched = pd.read_csv("dataset/MatchedData/ratings.csv")


In [5]:
brewery_rb = pd.read_csv("dataset/RateBeer/breweries.csv")
brewery_ba = pd.read_csv("dataset/BeerAdvocate/breweries.csv")
brewery_matched = pd.read_csv("dataset/MatchedData/breweries.csv")

In [6]:
# ID's of the beers should be unique after the merge operation. That's why we add 1000000 to
# id's of the beer advocate beers.

ba_id = beers_ba['beer_id'].values
rb_id = beers_rb['beer_id'].values
ba_id = ba_id + 1000000
if np.intersect1d(ba_id, rb_id).size > 0:
    print(np.intersect1d(ba_id, rb_id).size)
    
beers_ba['beer_id'] = beers_ba['beer_id'] + 1000000
ratings_ba['beer_id'] = ratings_ba['beer_id'] + 1000000
beers_matched['ba.5'][1:] = beers_matched['ba.5'][1:].astype(int) + 1000000
beers_matched['rb.4'][1:] = beers_matched['rb.4'][1:].astype(int)

In [7]:
# ID's of the breweries should be unique after the merge operation. That's why we add 1000000 to
# id's of the beer advocate breweries.

ba_id = brewery_ba['id'].values
rb_id = brewery_rb['id'].values
ba_id = ba_id + 1000000
if np.intersect1d(ba_id, rb_id).size > 0:
    print(np.intersect1d(ba_id, rb_id).size)

brewery_ba['id'] = brewery_ba['id'] + 1000000
beers_ba['brewery_id'] = beers_ba['brewery_id'] + 1000000
ratings_ba['brewery_id'] = ratings_ba['brewery_id'] + 1000000
brewery_matched['ba'][1:] = brewery_matched['ba'][1:].astype(int) + 1000000
brewery_matched['rb'][1:] = brewery_matched['rb'][1:].astype(int)


In [8]:
matched_rb_beers = beers_matched['rb.12'].unique()
all_rb_beers = beers_rb['style'].unique()

elements_not_in_matched = [element for element in all_rb_beers if element not in matched_rb_beers]

beers_rb = beers_rb[~beers_rb['style'].isin(elements_not_in_matched)]

In [9]:
ba_to_rb = []
for style_value in beers_matched['ba.14'].unique():
    # Filter the DataFrame for rows where 'style_column1' matches the current value
    filtered_rows = beers_matched[beers_matched['ba.14'] == style_value]
    
    # Find the most frequent match in 'style_column2'
    most_frequent_match = filtered_rows['rb.12'].mode().iloc[0]
    
    # Count the occurrences of the most frequent match
    match_count = filtered_rows[filtered_rows['rb.12'] == most_frequent_match].shape[0]
    
    # Append the results to the result DataFrame
    ba_to_rb.append({'style_column1': style_value,
                                  'most_frequent_match': most_frequent_match,
                                  'match_count': match_count})
ba_to_rb.pop(0)
ba_to_rb_df = pd.DataFrame(ba_to_rb)

In [10]:
rb_to_ba_styles = ba_to_rb_df.most_frequent_match.unique()
matched_rb_styles = beers_matched['rb.12'].unique()
elements_not_in_ba = [element for element in matched_rb_styles if element not in rb_to_ba_styles]

In [11]:
rb_to_ba_df = []
for style_value in elements_not_in_ba:
    # Filter the DataFrame for rows where 'style_column1' matches the current value
    filtered_rows = beers_matched[beers_matched['rb.12'] == style_value]
    
    # Find the most frequent match in 'style_column2'
    most_frequent_match = filtered_rows['ba.14'].mode().iloc[0]
    
    # Count the occurrences of the most frequent match
    match_count = filtered_rows[filtered_rows['ba.14'] == most_frequent_match].shape[0]
    
    # Append the results to the result DataFrame
    rb_to_ba_df.append({'style_column1': style_value,
                                  'most_frequent_match': most_frequent_match,
                                  'match_count': match_count})
rb_to_ba_df = pd.DataFrame(rb_to_ba_df)

In [12]:
rb_to_ba_dict = dict(zip(rb_to_ba_df['style_column1'], rb_to_ba_df['most_frequent_match']))
beers_matched['rb.12'] = beers_matched['rb.12'].replace(rb_to_ba_dict)
beers_rb['style'] = beers_rb['style'].replace(rb_to_ba_dict)

In [13]:
ba_to_rb_dict = dict(zip(ba_to_rb_df['style_column1'], ba_to_rb_df['most_frequent_match']))
beers_ba['style'] = beers_ba['style'].replace(ba_to_rb_dict)
beers_matched['rb.12'] = beers_matched['rb.12'].replace(ba_to_rb_dict)
beers_rb['style'] = beers_rb['style'].replace(ba_to_rb_dict)

In [14]:
matched_rb_ids = beers_matched['rb.4'][1:]
matched_ba_ids = beers_matched['ba.5'][1:]

In [15]:
rb_without_matched = beers_rb[~beers_rb['beer_id'].isin(matched_rb_ids)]
ba_without_matched = beers_ba[~beers_ba['beer_id'].isin(matched_ba_ids)]

In [16]:
columns_to_take = ['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style', 'abv']
column_name_mapping = {'rb.4': 'beer_id', 'rb.5': 'beer_name', 'rb.7': 'brewery_id', 'rb.8': 'brewery_name', 'rb.12': 'style', 'rb': 'abv'}
beers_matched.rename(columns = column_name_mapping, inplace = True)

merged_beers = pd.concat([rb_without_matched[columns_to_take], ba_without_matched[columns_to_take]], axis=0, ignore_index = True)
merged_beers = pd.concat([merged_beers, beers_matched.iloc[1:][columns_to_take]], axis=0, ignore_index = True)

merged_beers.to_csv("dataset/MergedData/beers.csv", index = False)


In [30]:
matched_ba_ids = brewery_matched['ba'][1:]
ba_without_matched = brewery_ba[~brewery_ba['id'].isin(matched_ba_ids)]
merged_breweries = pd.concat([brewery_rb, ba_without_matched], axis = 0, ignore_index = True)
merged_breweries.to_csv("dataset/MergedData/breweries.csv", index = False)

In [49]:
matched_ba_ids = users_matched['ba.4'][1:]
ba_without_matched = users_ba[~users_ba['user_id'].isin(matched_ba_ids)]
merged_users = pd.concat([users_rb, ba_without_matched], axis = 0, ignore_index = True)
merged_users.to_csv("dataset/MergedData/users.csv", index = False)